In [ ]:
import pandas as pd

In [ ]:
df_reviews = pd.read_csv("../data/archive/olist_order_reviews_dataset.csv")
df_orders = pd.read_csv("../data/archive/olist_orders_dataset.csv")
df_order_items = pd.read_csv("../data/archive/olist_order_items_dataset.csv")

In [ ]:
df = pd.merge(pd.merge(df_reviews, df_orders, on='order_id'), df_order_items, on='order_id')[['product_id', 'review_score']]

In [ ]:
grouped_by = df.groupby('product_id').agg({'review_score': ['count', 'mean']})

In [ ]:
grouped_by[grouped_by['review_score']['count'] > 2][grouped_by['review_score']['mean'] < 3]